In [2]:
import requests as r
import pandas as pd
import json
from PIL import Image
import geopandas as gpd
from shapely.geometry import Point
import pyproj

In [3]:
# In order to get a key, you need to register on the geoservices french website
# The key will not be used in the following code, however I put mine there if you intend to reuse the code for more advanced service
# You will then need to replace the second 'inspire' parameter by the key in the url request
api_key = "223kqajqdjl4hjf6e180ptge"

In [4]:
data = pd.read_csv('aeroports.csv')

In [5]:
# The first step is to get all coordinates and extract latitude and longitude in order to design a bounding box for the polygon

# The bouding box in the API is defined by 2 parameters : top-left cornerccoordinates, bottom_right corner coordinates
# We originally have airports coordinates as polygons (list of latitudes and longitudes), so we need to transform polygon coordinates to nearest fitting rectangle

# Top left corner coordinates are (max_latitude, min_longitude)
# Bottom right corner coordinates are (min_latitude, max_longitude)

data['coordinates_clean'] = data.apply(lambda x: x.geometry.replace('POLYGON','').replace('MULTI','')\
                                                    .strip().split(','), axis=1)

In [6]:
def get_all_coor(list_coor):
    all_lat = []
    all_lon = []
    for string in list_coor:
        string_clean = string.strip()
        lat = float(string_clean.split()[1].replace(')','').replace('(',''))
        lon = float(string_clean.split()[0].replace(')','').replace('(',''))
        all_lat.append(lat)
        all_lon.append(lon)
    return(all_lat, all_lon)

In [7]:
data['all_coor'] = data.apply(lambda x: get_all_coor(x.coordinates_clean), axis=1)

In [8]:
data['max_lat'] = data.apply(lambda x: str(max(x.all_coor[0])) ,axis=1)
data['min_lat'] = data.apply(lambda x: str(min(x.all_coor[0])),axis=1)
data['max_lon'] = data.apply(lambda x: str(max(x.all_coor[1])),axis=1)
data['min_lon'] = data.apply(lambda x: str(min(x.all_coor[1])),axis=1)

In [9]:
data['bbox'] = data.apply(lambda x: x.min_lat + ',' + x.min_lon + ',' + x.max_lat + ',' + x.max_lon, axis=1)

In [10]:
data['up_left'] = data.apply(lambda row: Point(float(row['max_lat']), float(row['min_lon'])), axis=1)
data['up_right'] = data.apply(lambda row: Point(float(row['max_lat']), float(row['max_lon'])), axis=1)
data['bot_left'] = data.apply(lambda row: Point(float(row['min_lat']), float(row['min_lon'])), axis=1)

C:\Users\paulc\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [11]:
import pyproj
geod = pyproj.Geod(ellps='WGS84')
_, _, distance = geod.inv(data['up_left'].iloc[1].x, data['up_left'].iloc[1].y, 
                          data['up_right'].iloc[1].x, data['up_right'].iloc[1].y)

In [12]:
data['width'] = data.apply(lambda row: geod.inv(  row['up_left'].x, row['up_left'].y, 
                                                  row['up_right'].x, row['up_right'].y)[2], axis=1)

In [13]:
data['height'] = data.apply(lambda row: geod.inv(  row['up_left'].x, row['up_left'].y, 
                                                   row['bot_left'].x, row['bot_left'].y)[2], axis=1)

In [14]:
url_start  = "https://wxs.ign.fr/ortho/geoportail/r/wms?LAYERS=HR.ORTHOIMAGERY.ORTHOPHOTOS&EXCEPTIONS=text/xml&" +\
            "FORMAT=image/png&SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&STYLES=&CRS=EPSG:4326&BBOX=" 
url_width  = " &WIDTH="
url_height = "&HEIGHT="

data['url_path'] = data.apply(lambda x: url_start + x.bbox + 
                              url_width + str(int(x.width * 5)) + 
                              url_height +str(int(x.height * 5)), axis=1)

In [15]:
data

,Unnamed: 0,ID,TOPONYME,geometry,coordinates_clean,all_coor,max_lat,min_lat,max_lon,min_lon,bbox,up_left,up_right,bot_left,width,height,url_path
0,0,BDCAEROD0000000017183202,aérodrome de cosne-sur-loire,POLYGON ((2.921117139197635 47.357152187886484...,"[((2.921117139197635 47.357152187886484, 2.92...","([47.357152187886484, 47.357335355326015, 47.3...",47.36294024661418,47.357152187886484,2.9233678458506147,2.9103049689323055,"47.357152187886484,2.9103049689323055,47.36294...",POINT (47.36294024661418 2.9103049689323055),POINT (47.36294024661418 2.9233678458506147),POINT (47.357152187886484 2.9103049689323055),1444.455714,643.498281,https://wxs.ign.fr/ortho/geoportail/r/wms?LAYE...
1,1,BDCAEROD0000000017183251,aérodrome de paray-le-monial,"POLYGON ((4.136658105500087 46.4630407689494, ...","[((4.136658105500087 46.4630407689494, 4.1339...","([46.4630407689494, 46.4642539524812, 46.46401...",46.470965857161254,46.4630407689494,4.137812175195705,4.127178560017682,"46.4630407689494,4.127178560017682,46.47096585...",POINT (46.470965857161254 4.127178560017682),POINT (46.470965857161254 4.137812175195705),POINT (46.4630407689494 4.127178560017682),1175.865615,879.944237,https://wxs.ign.fr/ortho/geoportail/r/wms?LAYE...
2,2,BDCAEROD0000000017183224,aérodrome de beaune-challanges,POLYGON ((4.892417384674342 47.000563649210896...,"[((4.892417384674342 47.000563649210896, 4.89...","([47.000563649210896, 47.0011376426943, 47.001...",47.014599884942115,47.000563649210896,4.89935923774398,4.887542055439518,"47.000563649210896,4.887542055439518,47.014599...",POINT (47.014599884942115 4.887542055439518),POINT (47.014599884942115 4.89935923774398),POINT (47.000563649210896 4.887542055439518),1306.771858,1556.862926,https://wxs.ign.fr/ortho/geoportail/r/wms?LAYE...
3,3,BDCAEROD0000000017183249,aérodrome de mâcon-charnay,"POLYGON ((4.794819533693073 46.28939045700893,...","[((4.794819533693073 46.28939045700893, 4.795...","([46.28939045700893, 46.29108558971598, 46.291...",46.300980323809526,46.28939045700893,4.79910504269433,4.791984581349047,"46.28939045700893,4.791984581349047,46.3009803...",POINT (46.300980323809526 4.791984581349047),POINT (46.300980323809526 4.79910504269433),POINT (46.28939045700893 4.791984581349047),787.395116,1285.698358,https://wxs.ign.fr/ortho/geoportail/r/wms?LAYE...
4,4,BDCAEROD0000000017183223,aérodrome de nuits-saint-georges,"POLYGON ((4.974093356762702 47.13970956430987,...","[((4.974093356762702 47.13970956430987, 4.962...","([47.13970956430987, 47.14333539405114, 47.144...",47.14464787647222,47.13970956430987,4.975278909241509,4.962333631680989,"47.13970956430987,4.962333631680989,47.1446478...",POINT (47.14464787647222 4.962333631680989),POINT (47.14464787647222 4.975278909241509),POINT (47.13970956430987 4.962333631680989),1431.522528,547.683601,https://wxs.ign.fr/ortho/geoportail/r/wms?LAYE...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,23,BDCAEROD0000000017183204,aérodrome d'aubigny-sur-nère,POLYGON ((2.391651403178498 47.476668054162495...,"[((2.391651403178498 47.476668054162495, 2.38...","([47.476668054162495, 47.47789453575687, 47.48...",47.4830324073134,47.476668054162495,2.400492567321413,2.3830027143191144,"47.476668054162495,2.3830027143191144,47.48303...",POINT (47.4830324073134 2.3830027143191144),POINT (47.4830324073134 2.400492567321413),POINT (47.476668054162495 2.3830027143191144),1933.961651,707.867964,https://wxs.ign.fr/ortho/geoportail/r/wms?LAYE...
657,24,BDCAEROD0000000017183227,aérodrome d'avord,POLYGON ((2.6318855105442087 47.03945339205281...,"[((2.6318855105442087 47.03945339205281, 2.63...","([47.03945339205281, 47.039499912930474, 47.03...",47.06797610411368,47.03945339205281,2.6587856193854837,2.5910003341935237,"47.03945339205281,2.5910003341935237,47.067976...",POINT (47.06797610411368 2.5910003341935237),POINT (47.06797610411368 2.6587856193854837),POINT (47.03945339205281 2.5910003341935237),7495.466690,3171.909486,https://wxs.ign.fr/ortho/ge

In [18]:
import os

In [19]:
os.getcwd()

'C:\\Users\\paulc\\colas-deep-learning\\open-data\\airports'

In [16]:
i = 0

for image_url in data['url_path']:
    response = r.get(image_url) ## Making a variable to get image.
    image_name = "image_folder/airport_" + str(i) + ".png" 
    file = open(image_name, "wb") ## Creates the file for image
    file.write(response.content) ## Saves file content
    file.close()
    i = i+1

KeyboardInterrupt: 

In [75]:
url_start = "https://wxs.ign.fr/ortho/geoportail/r/wms?LAYERS=HR.ORTHOIMAGERY.ORTHOPHOTOS&EXCEPTIONS=text/xml&" +\
            "FORMAT=image/png&SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&STYLES=&CRS=EPSG:4326&BBOX=" 
url_end = ""

data['url_path'] = data.apply(lambda x: url_start + x.bbox + url_end, axis=1)

response = r.get(data['url_path'].iloc[0]) ## Making a variable to get image.
image_name = "image_folder/airport_" + '663' + ".png" 
file = open(image_name, "wb") ## Creates the file for image
file.write(response.content) ## Saves file content
file.close()

In [76]:
data['url_path'].iloc[0]

'https://wxs.ign.fr/ortho/geoportail/r/wms?LAYERS=HR.ORTHOIMAGERY.ORTHOPHOTOS&EXCEPTIONS=text/xml&FORMAT=image/png&SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&STYLES=&CRS=EPSG:4326&BBOX=47.357152187886484,2.9103049689323055,47.36294024661418,2.9233678458506147'

In [ ]:
"https://wxs.ign.fr/ortho/geoportail/r/wms?LAYERS=HR.ORTHOIMAGERY.ORTHOPHOTOS&EXCEPTIONS=text/xml&" +\
            "SERVICE=WMS&VERSION=1.3.0&REQUEST=GetCapalities&BBOX="

In [128]:
os.chdir("/Users/paulc/colas-deep-learning/open-data/airports")

In [129]:
os.getcwd()

'C:\\Users\\paulc\\colas-deep-learning\\open-data\\airports'

In [133]:
import geopandas

In [134]:
data = geopandas.read_file("airports.geojson")

In [136]:
data['geometry']

0      POLYGON ((2.92112 47.35715, 2.92042 47.35734, ...
1      POLYGON ((4.13666 46.46304, 4.13392 46.46425, ...
2      POLYGON ((4.89242 47.00056, 4.89009 47.00114, ...
3      POLYGON ((4.79482 46.28939, 4.79551 46.29109, ...
4      POLYGON ((4.97409 47.13971, 4.96233 47.14334, ...
                             ...                        
607    POLYGON ((2.39165 47.47667, 2.38300 47.47789, ...
608    POLYGON ((2.63189 47.03945, 2.63123 47.03950, ...
609    POLYGON ((1.20515 47.66654, 1.20378 47.66835, ...
610    POLYGON ((1.68134 47.30980, 1.67842 47.31455, ...
611    POLYGON ((1.52031 48.45362, 1.51905 48.45565, ...
Name: geometry, Length: 612, dtype: geometry

In [139]:
data['geometry'].astype(str)

0      POLYGON ((2.921117139197635 47.357152187886484...
1      POLYGON ((4.136658105500087 46.4630407689494, ...
2      POLYGON ((4.892417384674342 47.000563649210896...
3      POLYGON ((4.794819533693073 46.28939045700893,...
4      POLYGON ((4.974093356762702 47.13970956430987,...
                             ...                        
607    POLYGON ((2.391651403178498 47.476668054162495...
608    POLYGON ((2.631885510544209 47.03945339205281,...
609    POLYGON ((1.205148216556766 47.666538795593894...
610    POLYGON ((1.681342103446778 47.309801410570195...
611    POLYGON ((1.520305023790417 48.453621171440545...
Name: geometry, Length: 612, dtype: object

In [140]:
data['coordinates_clean'] = data.apply(lambda x: x.geometry.replace('POLYGON','').replace('MULTI','')\
                                                    .strip().split(','), axis=1)

AttributeError: 'Polygon' object has no attribute 'astype'

In [142]:
os.getcwd()

'C:\\Users\\paulc\\colas-deep-learning\\open-data\\airports'

In [145]:
os.chdir("/Users/paulc/colas-deep-learning")
os.getcwd()

'C:\\Users\\paulc\\colas-deep-learning'

In [176]:
path = "/Users/paulc/colas-deep-learning/image_folder"

In [177]:
isExist = os.path.exists(path)
print(isExist)

False


In [166]:
import glob

In [180]:
if os.path.exists(path) == True:
    print('image_folder already exists')
    
else: 
    os.mkdir(path)

In [183]:
def clean_coordinates(data):
    """
    Input : Dataframe with extracted data from French airports database passed through data_preprocessing.py
        
    This function cleans the coordinates in order to remove useless characters.
    """
    data['geometry'] = data['geometry'].astype(str)
    data['coordinates_clean'] = data.apply(lambda x: x.geometry.replace('POLYGON','').replace('MULTI','').strip().split(','), axis=1)
        
    return data

In [217]:
### import requests as r
import pandas as pd
import json
from PIL import Image
import geopandas as gpd
from shapely.geometry import Point
import pyproj
import os
import yaml
from utils import clean_coordinates, get_all_coor, construct_bbox, get_width_height, get_url


# Load yaml config file.
os.chdir("/Users/paulc/colas-deep-learning")
with open("config.yml") as f:
    config = yaml.safe_load(f)

# Load data.
data = gpd.read_file(config["airport_data_path"])

# Clean Coordinates - Pass Geopandas Geometry format to string format.
data = clean_coordinates(data)

# Construction of the bbox rectangles that contain each airport. Bbox will be used during WMS request.
data = construct_bbox(data)

# Define the width and height needed for each airport photo in order to have a 20cm per pixel resolution.
data = get_width_height(data)

# Create the url request.
#data = get_url(data)

# Check if there is a folder named image_folder. If not, create one. 
if os.path.exists(config["working_directory_path"]+"image_folder") == True:
    print('image_folder already exists')
    
else: 
    os.mkdir(config["working_directory_path"]+"image_folder")


C:\Users\paulc\anaconda3\lib\site-packages\geopandas\geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
C:\Users\paulc\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\paulc\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\paulc\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_ob

image_folder already exists


In [218]:
data

,ID,TOPONYME,geometry,coordinates_clean,all_coor,max_lat,min_lat,max_lon,min_lon,bbox,up_left,up_right,bot_left,width,height
0,BDCAEROD0000000017183202,aérodrome de cosne-sur-loire,POLYGON ((2.921117139197635 47.357152187886484...,"[((2.921117139197635 47.357152187886484, 2.92...","([47.357152187886484, 47.357335355326015, 47.3...",47.36294024661418,47.357152187886484,2.923367845850615,2.910304968932306,"47.357152187886484,2.910304968932306,47.362940...",POINT (47.36294 2.91030),POINT (47.36294 2.92337),POINT (47.35715 2.91030),1444.455714,643.498281
1,BDCAEROD0000000017183251,aérodrome de paray-le-monial,"POLYGON ((4.136658105500087 46.4630407689494, ...","[((4.136658105500087 46.4630407689494, 4.1339...","([46.4630407689494, 46.4642539524812, 46.46401...",46.470965857161254,46.4630407689494,4.137812175195705,4.127178560017682,"46.4630407689494,4.127178560017682,46.47096585...",POINT (46.47097 4.12718),POINT (46.47097 4.13781),POINT (46.46304 4.12718),1175.865615,879.944237
2,BDCAEROD0000000017183224,aérodrome de beaune-challanges,POLYGON ((4.892417384674342 47.000563649210896...,"[((4.892417384674342 47.000563649210896, 4.89...","([47.000563649210896, 47.0011376426943, 47.001...",47.014599884942115,47.000563649210896,4.89935923774398,4.887542055439518,"47.000563649210896,4.887542055439518,47.014599...",POINT (47.01460 4.88754),POINT (47.01460 4.89936),POINT (47.00056 4.88754),1306.771858,1556.862926
3,BDCAEROD0000000017183249,aérodrome de mâcon-charnay,"POLYGON ((4.794819533693073 46.28939045700893,...","[((4.794819533693073 46.28939045700893, 4.795...","([46.28939045700893, 46.29108558971598, 46.291...",46.300980323809526,46.28939045700893,4.79910504269433,4.791984581349047,"46.28939045700893,4.791984581349047,46.3009803...",POINT (46.30098 4.79198),POINT (46.30098 4.79911),POINT (46.28939 4.79198),787.395116,1285.698358
4,BDCAEROD0000000017183223,aérodrome de nuits-saint-georges,"POLYGON ((4.974093356762702 47.13970956430987,...","[((4.974093356762702 47.13970956430987, 4.962...","([47.13970956430987, 47.14333539405114, 47.144...",47.14464787647222,47.13970956430987,4.975278909241509,4.962333631680989,"47.13970956430987,4.962333631680989,47.1446478...",POINT (47.14465 4.96233),POINT (47.14465 4.97528),POINT (47.13971 4.96233),1431.522528,547.683601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,BDCAEROD0000000017183204,aérodrome d'aubigny-sur-nère,POLYGON ((2.391651403178498 47.476668054162495...,"[((2.391651403178498 47.476668054162495, 2.38...","([47.476668054162495, 47.47789453575687, 47.48...",47.4830324073134,47.476668054162495,2.400492567321413,2.383002714319114,"47.476668054162495,2.383002714319114,47.483032...",POINT (47.48303 2.38300),POINT (47.48303 2.40049),POINT (47.47667 2.38300),1933.961651,707.867964
608,BDCAEROD0000000017183227,aérodrome d'avord,"POLYGON ((2.631885510544209 47.03945339205281,...","[((2.631885510544209 47.03945339205281, 2.631...","([47.03945339205281, 47.039499912930474, 47.03...",47.06797610411368,47.03945339205281,2.658785619385484,2.591000334193524,"47.03945339205281,2.591000334193524,47.0679761...",POINT (47.06798 2.59100),POINT (47.06798 2.65879),POINT (47.03945 2.59100),7495.466690,3171.909486
609,BDCAEROD0000000017183177,aéroport de blois-le breuil,POLYGON ((1.205148216556766 47.666538795593894...,"[((1.205148216556766 47.666538795593894, 1.20...","([47.666538795593894, 47.668351695909045, 47.6...",47.684722435732546,47.666538795593894,1.219675679724876,1.197095314628524,"47.666538795593894,1.197095314628524,47.684722...",POINT (47.68472 1.19710),POINT (47.68472 1.21968),POINT (47.66654 1.19710),2496.818669,2023.754725
610,BDCAEROD0000000017183207,aérodrome de romorantin-pruniers,POLYGON ((1.681342103446778 47.309801410570195...,"[((1.681342103446778 47.309801410570195, 1.67...","([47.309801410570195, 47.314547331269566, 47.3...",47.32594671739858,47.30938267810873,1.699564479866769,1.67817930056178,"47.30938267810873,1.67817930056178,47.32594671...",POINT (47.32